In [1]:
from bokeh.plotting import figure, save, show, output_file, output_notebook
from bokeh.models import GeoJSONDataSource, ColumnDataSource, Range1d, HoverTool, LogColorMapper, BoxZoomTool
from bokeh.resources import CDN
from bokeh.layouts import row, widgetbox
from bokeh.palettes import Blues6 as palette
from bokeh.models.widgets import RadioButtonGroup, Slider, Dropdown
from bokeh.models.callbacks import CustomJS
import geopandas as gpd
import numpy as np
import pandas as pd

In [2]:
df = pd.read_excel("ex.xlsx", encoding='utf-8')
#PATH = r"mapdata_t/TOWN_MOI_1071031.shp"
PATH = r"mapdata_c/COUNTY_MOI_1070516.shp"
gdf = gpd.read_file(PATH, encoding='utf-8') #全台灣村里界圖
gdf = gdf.to_crs({'init' :'epsg:4326'})
palette.reverse()

In [3]:
gdf.columns

Index(['COUNTYID', 'COUNTYCODE', 'COUNTYNAME', 'COUNTYENG', 'geometry'], dtype='object')

In [4]:
gdf.head()

,COUNTYID,COUNTYCODE,COUNTYNAME,COUNTYENG,geometry
0,Z,09007,連江縣,Lienchiang County,(POLYGON ((119.9644519140001 25.94552317400007...
1,G,10002,宜蘭縣,Yilan County,(POLYGON ((121.9597084550001 24.84493697000005...
2,N,10007,彰化縣,Changhua County,"POLYGON ((120.4565526600001 24.20718620500008,..."
3,M,10008,南投縣,Nantou County,"POLYGON ((121.2708644380001 24.23660985400005,..."
4,P,10009,雲林縣,Yunlin County,"(POLYGON ((120.081077282 23.52412216400006, 12..."


In [5]:
ndf = pd.merge(gdf, df)
villages_shp = gpd.GeoDataFrame(ndf)

In [6]:
def cartesian(x,n): 
    return np.vstack(np.array([np.array(np.meshgrid(*i)).T.reshape(-1,n) for i in x.values]))

In [7]:
villages_shp.head()
#villages_shp.shape

,COUNTYID,COUNTYCODE,COUNTYNAME,COUNTYENG,geometry,RATE7,RATE8,RATE9,RATE10,RATE11,RATE12,RATE13,RATE14,RATE15,RATE16
0,Z,09007,連江縣,Lienchiang County,(POLYGON ((119.9644519140001 25.94552317400007...,43.22,41.99,42.40,38.74,36.87,32.12,10.50,13.32,13.21,33.59
1,G,10002,宜蘭縣,Yilan County,(POLYGON ((121.9597084550001 24.84493697000005...,39.66,38.57,38.60,38.91,35.33,31.02,26.66,17.10,17.92,28.94
2,N,10007,彰化縣,Changhua County,"POLYGON ((120.4565526600001 24.20718620500008,...",42.39,40.06,40.80,41.91,37.94,33.45,24.70,15.91,16.63,30.77
3,M,10008,南投縣,Nantou County,"POLYGON ((121.2708644380001 24.23660985400005,...",43.82,41.87,41.90,43.09,38.83,33.57,23.05,15.08,16.01,32.51
4,P,10009,雲林縣,Yunlin County,"(POLYGON ((120.081077282 23.52412216400006, 12...",36.53,34.52,35.23,37.33,33.57,29.15,23.84,14.35,15.08,27.18


In [8]:
villages_shp = gpd.GeoDataFrame(cartesian(villages_shp, 5+10),columns=villages_shp.columns)

In [9]:
villages_shp.head(6)

,COUNTYID,COUNTYCODE,COUNTYNAME,COUNTYENG,geometry,RATE7,RATE8,RATE9,RATE10,RATE11,RATE12,RATE13,RATE14,RATE15,RATE16
0,Z,09007,連江縣,Lienchiang County,"POLYGON ((119.9644519140001 25.94552317400007,...",43.22,41.99,42.4,38.74,36.87,32.12,10.5,13.32,13.21,33.59
1,Z,09007,連江縣,Lienchiang County,"POLYGON ((119.9646932640001 25.94687440200005,...",43.22,41.99,42.4,38.74,36.87,32.12,10.5,13.32,13.21,33.59
2,Z,09007,連江縣,Lienchiang County,"POLYGON ((119.9643200390001 25.94638165700007,...",43.22,41.99,42.4,38.74,36.87,32.12,10.5,13.32,13.21,33.59
3,Z,09007,連江縣,Lienchiang County,"POLYGON ((119.9848039630001 25.96429508900007,...",43.22,41.99,42.4,38.74,36.87,32.12,10.5,13.32,13.21,33.59
4,Z,09007,連江縣,Lienchiang County,"POLYGON ((119.9853612840001 25.96461429900006,...",43.22,41.99,42.4,38.74,36.87,32.12,10.5,13.32,13.21,33.59
5,Z,09007,連江縣,Lienchiang County,"POLYGON ((119.9893897520001 25.97037512200006,...",43.22,41.99,42.4,38.74,36.87,32.12,10.5,13.32,13.21,33.59


In [10]:
def getPolyCoords(row, geom, coord_type):
    """Returns the coordinates ('x' or 'y') of edges of a Polygon exterior"""

    # Parse the exterior of the coordinate
    exterior = row[geom].exterior

    if coord_type == 'x':
        # Get the x coordinates of the exterior
        return list( exterior.coords.xy[0] )
    elif coord_type == 'y':
        # Get the y coordinates of the exterior
        return list( exterior.coords.xy[1] )

In [11]:
villages_shp['x'] = villages_shp.apply(getPolyCoords, geom='geometry', coord_type='x', axis=1)
villages_shp['y'] = villages_shp.apply(getPolyCoords, geom='geometry', coord_type='y', axis=1)

In [12]:
villages_shp.head(6)

,COUNTYID,COUNTYCODE,COUNTYNAME,COUNTYENG,geometry,RATE7,RATE8,RATE9,RATE10,RATE11,RATE12,RATE13,RATE14,RATE15,RATE16,x,y
0,Z,09007,連江縣,Lienchiang County,"POLYGON ((119.9644519140001 25.94552317400007,...",43.22,41.99,42.4,38.74,36.87,32.12,10.5,13.32,13.21,33.59,"[119.96445191400005, 119.96426754100003, 119.9...","[25.945523174000073, 25.945488368000042, 25.94..."
1,Z,09007,連江縣,Lienchiang County,"POLYGON ((119.9646932640001 25.94687440200005,...",43.22,41.99,42.4,38.74,36.87,32.12,10.5,13.32,13.21,33.59,"[119.96469326400006, 119.96469944500005, 119.9...","[25.94687440200005, 25.946863776000043, 25.946..."
2,Z,09007,連江縣,Lienchiang County,"POLYGON ((119.9643200390001 25.94638165700007,...",43.22,41.99,42.4,38.74,36.87,32.12,10.5,13.32,13.21,33.59,"[119.96432003900009, 119.96411690100001, 119.9...","[25.946381657000074, 25.946294144000035, 25.94..."
3,Z,09007,連江縣,Lienchiang County,"POLYGON ((119.9848039630001 25.96429508900007,...",43.22,41.99,42.4,38.74,36.87,32.12,10.5,13.32,13.21,33.59,"[119.9848039630001, 119.98479340300003, 119.98...","[25.964295089000075, 25.964292340000043, 25.96..."
4,Z,09007,連江縣,Lienchiang County,"POLYGON ((119.9853612840001 25.96461429900006,...",43.22,41.99,42.4,38.74,36.87,32.12,10.5,13.32,13.21,33.59,"[119.98536128400006, 119.98530636400007, 119.9...","[25.964614299000065, 25.964601334000065, 25.96..."
5,Z,09007,連江縣,Lienchiang County,"POLYGON ((119.9893897520001 25.97037512200006,...",43.22,41.99,42.4,38.74,36.87,32.12,10.5,13.32,13.21,33.59,"[119.9893897520001, 119.98963574700008, 119.98...","[25.970375122000064, 25.970334440000045, 25.97..."


In [13]:
town_xs = villages_shp.x
town_ys = villages_shp.y
#town_name = villages_shp.COUNTYNAME + ", " +  villages_shp.TOWNNAME
town_name = villages_shp.COUNTYNAME

rate_7 = villages_shp.RATE7
rate_8 = villages_shp.RATE8
rate_9 = villages_shp.RATE9
rate_10 = villages_shp.RATE10
rate_11 = villages_shp.RATE11
rate_12 = villages_shp.RATE12
rate_13 = villages_shp.RATE13
rate_14 = villages_shp.RATE14
rate_15 = villages_shp.RATE15
rate_16 = villages_shp.RATE16
rate_default = villages_shp.RATE7

In [14]:
#villages_shp_source = GeoJSONDataSource(geojson=villages_shp.to_json())
source = ColumnDataSource(data=dict(
    x=town_xs,
    y=town_ys,
    name=town_name,
    rate7=rate_7,
    rate8=rate_8,
    rate9=rate_9,
    rate10=rate_10,
    rate11=rate_11,
    rate12=rate_12,
    rate13=rate_13,
    rate14=rate_14,
    rate15=rate_15,
    rate16=rate_16,
    show=rate_7
))

TOOLTIPS = [
    #("Latitude", "$x"),
    #("Longitude", "$y"),
    ("County", "@name"), 
    ("Rate", "@show{%0.2f}%")
]
color_mapper = LogColorMapper(palette=palette)# Color map
#HoverTool.mode="vline"

In [15]:
# Initialize our plot figure
p = figure(title="Taiwan Map", plot_width=1200, plot_height=900,
          tools="wheel_zoom,reset",
          x_axis_label="Latitude", y_axis_label="Longitude")
#p.axis.visible=False
p.add_tools(HoverTool(
    tooltips=TOOLTIPS,
    
    formatters={
        'show' : 'printf',   # use 'printf' formatter for 'rate' field
                             # use default 'numeral' formatter for other fields
    },
    # display a tooltip whenever the cursor is vertically in line with a glyph
    #mode='vline'
))

#p.grid.grid_line_color = None
p.hover.point_policy = "follow_mouse"
'''
zoom_overlay = p.select_one(BoxZoomTool).overlay

zoom_overlay.line_color = "olive"
zoom_overlay.line_width = 5
zoom_overlay.line_dash = "solid"
zoom_overlay.fill_color = None
'''

'\nzoom_overlay = p.select_one(BoxZoomTool).overlay\n\nzoom_overlay.line_color = "olive"\nzoom_overlay.line_width = 5\nzoom_overlay.line_dash = "solid"\nzoom_overlay.fill_color = None\n'

In [16]:
# Add the lines to the map from our GeoJSONDataSource -object
#p.multi_line('x', 'y', source=source, color='black', line_width=2)
color = {'field': 'show', 'transform': color_mapper}
p.patches('x', 'y', source=source,
          fill_color=color,
          fill_alpha=0.7, line_color="white", line_width=0.5)

GlyphRenderer(id='1035', ...)

In [17]:
# add callback to control 
def callback(p=p, source=source, window=None):
    data = source.data
    f = cb_obj.value
    l = cb_obj.label
    rate7 = data['rate7']
    rate8 = data['rate8']
    rate9 = data['rate9']
    rate10 = data['rate10']
    rate11 = data['rate11']
    rate12 = data['rate12']
    rate13 = data['rate13']
    rate14 = data['rate14']
    rate15 = data['rate15']
    rate16 = data['rate16']
    
    show = data['show']
    
    def switch_rate(num):
        switcher = {
            7 : rate7,
            8 : rate8,
            9 : rate9,
            10 : rate10,
            11 : rate11,
            12 : rate12,
            13 : rate13,
            14 : rate14,
            15 : rate15,
            16 : rate16
        }
        return switcher.get(num)
    
    for i in range(len(show)):
        show[i] = switch_rate(f)[i]
                
    cb_obj.label = "Referendum No." + str(f)
    source.change.emit()

menu = [("7", "7"), None,
        ("8", "8"), None,
       ("9", "9"), None,
       ("10", "10"), None,
       ("11", "11"), None,
       ("12", "12"), None,
       ("13", "13"), None,
       ("14", "14"), None,
       ("15", "15"), None,
       ("16", "16"), None]
dropdown = Dropdown(label="Referendum", menu=menu, button_type="primary")
dropdown.js_on_change('value', CustomJS.from_py_func(callback))

In [18]:
output_file("taiwanmap.html")
#output_notebook()

In [19]:
layout = row(p, widgetbox(dropdown))
#show(layout)
save(layout, title="taiwanmap")

'D:\\Desktop\\MS-Project\\taiwanmap.html'